--------------------------------
# <center>- Spark Project - SD 701 - Data Mining-</center>

* https://spark.apache.org/docs/2.1.0/ml-classification-regression.html
* https://spark.apache.org/docs/latest/mllib-collaborative-filtering.html
* https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html

----------------------------
* https://medium.com/@patelneha1495/recommendation-system-in-python-using-als-algorithm-and-apache-spark-27aca08eaab3
* https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-2-alternating-least-square-als-matrix-4a76c58714a1

----------------------------------

In [41]:
import pyspark
#from pyspark import SparkConf, SparkContext

#from pyspark.ml.classification import LogisticRegression
#from pyspark.ml.regression import LinearRegression

from pyspark.sql import Row, SparkSession

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS


### 1. Initialisation de la session spark

In [2]:
# set up environment
#conf = SparkConf() \
#    .setAppName("MovieLensALS") \
#    .set("spark.executor.memory", "2g")
#sc = SparkContext(conf=conf)

spark = SparkSession.builder.appName('Recommendation_system').getOrCreate()

### 2. Chargement et préparation des données d'évaluations des films

In [3]:
path_data = "/home/p5hngk/Downloads/GitHub/SD_701---Data_Mining/ml-latest-small"

df_ratings = spark.read.format("csv").option("header", "true").load(path_data+"/ratings.csv")
df_ratings.show(10)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
+------+-------+------+---------+
only showing top 10 rows



In [6]:
df_ratings1 = df_ratings.select(df_ratings['userId'], df_ratings['movieId'], df_ratings['rating'])
df_ratings1.show(10)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
+------+-------+------+
only showing top 10 rows



In [7]:
df_ratings1.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)



Avant de réaliser un modèle ALS, il faut que toutes nos données soient au format integer pour pouvoir réaliser les calculs, ce qui n'est pas le cas actuellement. Occupons-nous donc dans un premier temps de changer cela .

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [11]:
from pyspark.sql.types import IntegerType

df_ratings1 = df_ratings1.withColumn("userId", df_ratings1["userId"].cast(IntegerType())) \
                .withColumn("movieId", df_ratings1["movieId"].cast(IntegerType())) \
                .withColumn("rating", df_ratings1["rating"].cast(IntegerType()))

df_ratings1.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: integer (nullable = true)



### 3. Création des jeux d'entraînement et de test

In [13]:
(training,test) = df_ratings1.randomSplit([0.8, 0.2])

### 4. Création du modèle ALS et entraînement du modèle

In [83]:
als = ALS(maxIter=5, regParam=0.15, rank=25, userCol = "userId", itemCol = "movieId", ratingCol = "rating", coldStartStrategy = "drop", nonnegative=True)
model = als.fit(training)

### 5. Evaluation du modèle

In [86]:
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating", predictionCol = "prediction")
predictions = model.transform(test)
rmse = evaluator.evaluate(predictions)

In [87]:
print(f"RMSE = {round(rmse,3)}")
predictions.show(10)

RMSE = 0.912
+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   602|    471|     4| 3.1934433|
|   462|    471|     2| 2.7162755|
|   217|    471|     2| 2.7824652|
|   171|    471|     3|  4.175868|
|   287|    471|     4| 2.2260783|
|   469|    471|     5| 3.3332288|
|   307|    833|     1| 0.8265029|
|   177|   1088|     3|   3.22892|
|   554|   1088|     5| 3.3106422|
|   286|   1088|     3| 2.8980076|
+------+-------+------+----------+
only showing top 10 rows



In [25]:
predictions.describe().show()

+-------+------------------+------------------+-----------------+------------------+
|summary|            userId|           movieId|           rating|        prediction|
+-------+------------------+------------------+-----------------+------------------+
|  count|             19335|             19335|            19335|             19335|
|   mean|   324.51874838376|17180.324385828808|3.369847426945953|3.1990105139020804|
| stddev|181.40932579600593|32340.785112503912|1.085566007162708|0.7425755555118295|
|    min|                 1|                 1|                0|               0.0|
|    max|               610|            187595|                5|          5.506267|
+-------+------------------+------------------+-----------------+------------------+



In [32]:
predictions.orderBy(predictions.userId.asc()).show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|   3273|     5| 2.7050326|
|     1|    441|     4| 4.5551343|
|     1|    163|     5| 4.0466523|
|     1|    590|     4| 4.2134485|
|     1|   3740|     4| 4.7146983|
|     1|   3489|     4| 3.5443125|
|     1|   2492|     4| 3.2470827|
|     1|   1089|     5|  4.778551|
|     1|   2193|     4| 4.6384244|
|     1|   1240|     5| 4.3902125|
|     1|   2985|     4| 3.9428837|
|     1|   2414|     3| 3.9190807|
|     1|   3439|     4| 3.1636713|
|     1|   1500|     4| 4.3516064|
|     1|   2640|     4|  4.639884|
|     1|   1777|     4| 4.0192113|
|     1|   2542|     5|  4.635328|
|     1|   1927|     5| 4.4290094|
|     1|    367|     4|  3.847385|
|     1|      3|     4|  3.852513|
+------+-------+------+----------+
only showing top 20 rows



In [ ]:
from pyspark.sql import SQLContext

sqlContext = SQLContext(spark)
predictions.registerTempTable("predictions_table")

In [56]:
sqlContext.sql('SELECT * FROM predictions_table WHERE userId = 1 ORDER BY movieId ASC').show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|      3|     4|  3.852513|
|     1|     50|     5| 4.5171766|
|     1|    163|     5| 4.0466523|
|     1|    367|     4|  3.847385|
|     1|    441|     4| 4.5551343|
|     1|    457|     5| 4.4773464|
|     1|    480|     4|  4.376313|
|     1|    527|     5|  4.645906|
|     1|    590|     4| 4.2134485|
|     1|   1042|     4| 3.8467429|
|     1|   1080|     5|  4.726887|
|     1|   1089|     5|  4.778551|
|     1|   1097|     5| 4.4196105|
|     1|   1213|     5| 4.7464733|
|     1|   1240|     5| 4.3902125|
|     1|   1282|     5| 4.4678564|
|     1|   1408|     3| 4.8702917|
|     1|   1500|     4| 4.3516064|
|     1|   1777|     4| 4.0192113|
|     1|   1805|     4| 3.6425023|
+------+-------+------+----------+
only showing top 20 rows



### 6. Amélioration du modèle

Pouvons-nous améliorer notre modèle avec de meilleurs hyperparamètres ? Nous allons regarder ici l'influence des paramètres de régularisations (`regParams`) et le nombre de features (`ranks`) à utiliser pour notre modèle. Nous chercherons ici à minimiser le risque moyen quadratique (***RMSE***).

In [78]:
def tune_ALS(train_data, validation_data, maxIter, regParams, ranks):
    """
    grid search function to select the best model based on RMSE of
    validation data
    Parameters
    ----------
    train_data: spark DF with columns ['userId', 'movieId', 'rating']
    
    validation_data: spark DF with columns ['userId', 'movieId', 'rating']
    
    maxIter: int, max number of learning iterations
    
    regParams: list of float, one dimension of hyper-param tuning grid
    
    ranks: list of float, one dimension of hyper-param tuning grid
    
    Return
    ------
    The best fitted ALS model with lowest RMSE score on validation data
    """
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in regParams:
            # get ALS model
            als = ALS(userCol = "userId", itemCol = "movieId", ratingCol = "rating", coldStartStrategy = "drop", nonnegative=True).setMaxIter(maxIter).setRank(rank).setRegParam(reg)
            # train ALS model
            model = als.fit(train_data)
            # evaluate the model by computing the RMSE on the validation data
            predictions = model.transform(validation_data)
            evaluator = RegressionEvaluator(metricName="rmse",
                                            labelCol="rating",
                                            predictionCol="prediction")
            rmse = evaluator.evaluate(predictions)
            print('{} latent factors and regularization = {}: '
                  'validation RMSE is {}'.format(rank, reg, rmse))
            if rmse < min_error:
                min_error = rmse
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and '
          'regularization = {}'.format(best_rank, best_regularization))
    return best_model

In [88]:
regParams = [0.10, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2]
ranks = [15, 20, 25, 30]

tune_ALS(training, test, 10, regParams, ranks)

15 latent factors and regularization = 0.1: validation RMSE is 0.9194962266285418
15 latent factors and regularization = 0.11: validation RMSE is 0.9158210040273436
15 latent factors and regularization = 0.12: validation RMSE is 0.9135584134461238
15 latent factors and regularization = 0.13: validation RMSE is 0.9122796084347585
15 latent factors and regularization = 0.14: validation RMSE is 0.9118776751844955
15 latent factors and regularization = 0.15: validation RMSE is 0.9121690831477112
15 latent factors and regularization = 0.16: validation RMSE is 0.9130609447437331
15 latent factors and regularization = 0.17: validation RMSE is 0.9144633840148606
15 latent factors and regularization = 0.18: validation RMSE is 0.916332021519337
15 latent factors and regularization = 0.19: validation RMSE is 0.9185967659831803
15 latent factors and regularization = 0.2: validation RMSE is 0.9212117989211674
20 latent factors and regularization = 0.1: validation RMSE is 0.9218943653846164
20 laten

ALS_9818eab060a0

### 7. Création du modèle avec les meilleurs hyperparamètres

In [98]:
best_als = ALS(maxIter=10, regParam=0.14, rank=25, userCol = "userId", itemCol = "movieId", ratingCol = "rating", coldStartStrategy = "drop", nonnegative=True)
best_model = best_als.fit(training)

In [99]:
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "rating", predictionCol = "prediction")
predictions = best_model.transform(test)
rmse = evaluator.evaluate(predictions)

In [100]:
print(f"RMSE = {round(rmse,3)}")
predictions.show(10)

RMSE = 0.909
+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   602|    471|     4|  3.439806|
|   462|    471|     2| 2.7085047|
|   217|    471|     2| 2.7092218|
|   171|    471|     3|  4.147817|
|   287|    471|     4| 2.2893267|
|   469|    471|     5| 3.3155427|
|   307|    833|     1| 0.9741007|
|   177|   1088|     3| 3.3184469|
|   554|   1088|     5| 3.4799078|
|   286|   1088|     3| 2.9636164|
+------+-------+------+----------+
only showing top 10 rows



### 8. Réalisation de recommandations

In [101]:
# Generate top 10 movie recommendations for each user
userRecs = best_model.recommendForAllUsers(10).show(10)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[7842, 4.5132], ...|
|   463|[[7842, 5.144905]...|
|   496|[[7842, 4.533911]...|
|   148|[[8477, 4.655421]...|
|   540|[[7842, 5.5218005...|
|   392|[[8477, 4.842426]...|
|   243|[[67618, 5.862327...|
|    31|[[33649, 5.740833...|
|   516|[[4429, 4.7700167...|
|   580|[[7842, 5.212381]...|
+------+--------------------+
only showing top 10 rows



In [102]:
# Generate top 10 user recommendations for each movie
movieRecs = best_model.recommendForAllItems(10).show(10)

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|   1580|[[53, 4.7668843],...|
|   4900|[[53, 4.532554], ...|
|   5300|[[53, 4.2717924],...|
|   6620|[[191, 4.574413],...|
|   7340|[[53, 3.979438], ...|
|  32460|[[53, 5.301957], ...|
|  54190|[[53, 5.637684], ...|
|    471|[[53, 4.8591766],...|
|   1591|[[37, 3.6290343],...|
|   1342|[[171, 3.5377822]...|
+-------+--------------------+
only showing top 10 rows



In [ ]:
# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = best_model.recommendForUserSubset(users, 10)

In [ ]:
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = best_model.recommendForItemSubset(movies, 10)

------------------------------------
-----------------------------------